In [17]:
# Library for creating data paths
import os

# Library for randomly selecting data points
import random

# Library for performing numerical computations
import numpy as np

# Library for creating and showing plots
import matplotlib.pyplot as plt

# Library for reading and showing images
import matplotlib.image as mpimg
import tensorflow as tf
# Importing all the required sub-modules from Keras
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

In [18]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


In [19]:
train_images.shape
test_images.shape
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
train_labels.shape
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [20]:
cnn_model = Sequential()
cnn_model.add(Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1), padding = 'same'))
cnn_model.add(MaxPooling2D(2,2))

cnn_model.add(Conv2D(32, (3,3), activation='relu', padding = 'same'))
cnn_model.add(MaxPooling2D(2,2))
cnn_model.add(BatchNormalization())

cnn_model.add(Conv2D(16, (3,3), activation='relu', padding = 'same'))
cnn_model.add(Flatten(input_shape=(28,28)))


cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dropout(0.25))

cnn_model.add(Dense(16, activation='relu'))

cnn_model.add(Dense(10, activation='softmax'))

In [21]:
cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics = ['accuracy'])
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 7, 32)          128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 16)          4624      
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)              

In [22]:
random.seed(1)

In [23]:
cnn_model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1143 - accuracy: 0.7759
Epoch 2/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0683 - accuracy: 0.8664
Epoch 3/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0597 - accuracy: 0.8846
Epoch 4/5
1875/1875 [==============================] - 31s 17ms/step - loss: 0.0546 - accuracy: 0.8941
Epoch 5/5
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0507 - accuracy: 0.9015


In [25]:
test_loss, test_acc = cnn_model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 2s - loss: 0.0501 - accuracy: 0.9051

Test accuracy: 0.9050999879837036


In [26]:
vgg_model = VGG16(weights='imagenet', include_top = False, input_shape = (224,224,3))
vgg_model.summary()
for layer in vgg_model.layers:
    layer.trainable = False

58900480/58889256 [==============================] - 43s 1us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____________________________________________________________

In [30]:
new_model = Sequential()

# Adding the convolutional part of the VGG16 model from above
new_model.add(vgg_model)

# Flattening the output of the VGG16 model because it is from a convolutional layer
new_model.add(Flatten())

# Adding a dense output layer
new_model.add(Dense(32, activation='relu'))
new_model.add(Dense(16, activation='relu'))
new_model.add(Dense(16, activation='relu'))
new_model.add(Dense(10, activation='softmax'))

In [31]:
new_model.compile(loss="binary_crossentropy", optimizer="adam", metrics = ['accuracy'])
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                802848    
_________________________________________________________________
dense_13 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_14 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_15 (Dense)             (None, 10)                170       
Total params: 15,518,506
Trainable params: 803,818
Non-trainable params: 14,714,688
____________________________________